# About the Data

* The data was taken from 
the csv file complaints.csv

* The Input text is "Consumer complaint narrative." which is preprocessed to predict the "Product".

* The complaints are for the products:<br>

  1. Banking Services
  2. Card Services
  3. Credit Reporting
  4. Debt Collection
  5. Loans
  6. Mortgage

* The data cleaning was done using gensim library

* A sample of the Data 'sample_df' which has 10000 entries of each of the 'product' counts is created for fine-tuning DistilBERT.



##Memory Allocated

In [ ]:
#Display Allocated Memory in Colab
!free -h --si | awk  '/Mem:/{print $2}'

26G


## Google Drive access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# write the appropriate paths to retrieve the data and store results 

saved_path = '/content/drive/MyDrive/EY_Internship/Consumer_Complaints/'
data_path = '/content/drive/MyDrive/EY_Internship/Consumer_Complaints/Raw_Data/complaints.csv'

# Loading the dataset

In [ ]:
#Set Paramenters 
fix_seed = 42

In [ ]:
#Load the data
import pandas as pd

raw_data= pd.read_csv(data_path, dtype={"Consumer complaint narrative": "string", "Consumer consent provided?": "string", "Timely response?":"string"})
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1922572 entries, 0 to 1922571
Data columns (total 18 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Date received                 object
 1   Product                       object
 2   Sub-product                   object
 3   Issue                         object
 4   Sub-issue                     object
 5   Consumer complaint narrative  string
 6   Company public response       object
 7   Company                       object
 8   State                         object
 9   ZIP code                      object
 10  Tags                          object
 11  Consumer consent provided?    string
 12  Submitted via                 object
 13  Date sent to company          object
 14  Company response to consumer  object
 15  Timely response?              string
 16  Consumer disputed?            object
 17  Complaint ID                  int64 
dtypes: int64(1), object(14), string(3)
memory 

In [ ]:
#Print the Product counts in raw_data
raw_data['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    612582
Debt collection                                                                 329695
Mortgage                                                                        317836
Credit reporting                                                                140432
Credit card or prepaid card                                                      99278
Credit card                                                                      89190
Bank account or service                                                          86206
Checking or savings account                                                      79836
Student loan                                                                     60258
Consumer Loan                                                                    31604
Money transfer, virtual currency, or money service                               22021
Vehicle loan or lease                      

In [ ]:
#Create dataframe df1 with the columns 'Product' and Consumer complaint narrative' from raw_data
df1 = raw_data[['Product', 'Consumer complaint narrative']]
#Rename the columns 'Product' as 'product' and 'Consumer complaint narrative' as 'complaint'
df1.columns = ['product', 'complaint']
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1922572 entries, 0 to 1922571
Data columns (total 2 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   product    object
 1   complaint  string
dtypes: object(1), string(1)
memory usage: 29.3+ MB


In [ ]:
#Check for null values in the dataframe df1
df1.isnull().sum()

product            0
complaint    1270120
dtype: int64

In [ ]:
#Drop null values in the dataframe df1
df1 = df1[df1['complaint'].notnull() & df1['product'].notnull()]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 652452 entries, 0 to 1922571
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   product    652452 non-null  object
 1   complaint  652452 non-null  string
dtypes: object(1), string(1)
memory usage: 14.9+ MB


In [ ]:
# Print the counts of unique values in 'product' 
df1['product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    233854
Debt collection                                                                 130474
Mortgage                                                                         72924
Credit card or prepaid card                                                      47763
Credit reporting                                                                 31588
Checking or savings account                                                      27716
Student loan                                                                     27427
Credit card                                                                      18838
Bank account or service                                                          14885
Money transfer, virtual currency, or money service                               12212
Vehicle loan or lease                                                            11664
Consumer Loan                              

In [ ]:
# Create a column to store the original_product
df1['original_product'] = df1['product'].copy()

df1 = df1.replace({'product':
             {'Credit reporting, credit repair services, or other personal consumer reports': 'Credit Reporting',
              'Debt collection': 'Debt Collection',
              'Credit reporting': 'Credit Reporting',
              'Credit card': 'Card Services',
              'Bank account or service': 'Banking Services',
              'Credit card or prepaid card': 'Card Services',
              'Student loan': 'Loans',
              'Checking or savings account': 'Banking Services',
              'Consumer Loan': 'Loans',
              'Vehicle loan or lease': 'Loans',
              'Money transfer, virtual currency, or money service': 'Banking Services',
              'Payday loan, title loan, or personal loan': 'Loans',
              'Payday loan': 'Loans',
              'Money transfers': 'Banking Services',
              'Prepaid card': 'Card Services',
              'Other financial service': 'Other',
              'Virtual currency': 'Banking Services'}
            })
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 652452 entries, 0 to 1922571
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   product           652452 non-null  object
 1   complaint         652452 non-null  string
 2   original_product  652452 non-null  object
dtypes: object(2), string(1)
memory usage: 19.9+ MB


In [ ]:
#Print the counts of unique values in 'product' 
df1['product'].value_counts()

Credit Reporting    265442
Debt Collection     130474
Mortgage             72924
Card Services        68051
Loans                58943
Banking Services     56326
Other                  292
Name: product, dtype: int64

In [ ]:
#Drop entries having 'product' 'Other' as it is under-represented
df1 = df1[df1['product'] != 'Other']
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 652160 entries, 0 to 1922571
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   product           652160 non-null  object
 1   complaint         652160 non-null  string
 2   original_product  652160 non-null  object
dtypes: object(2), string(1)
memory usage: 19.9+ MB


In [ ]:
#Print the products counts
df1['product'].value_counts()

Credit Reporting    265442
Debt Collection     130474
Mortgage             72924
Card Services        68051
Loans                58943
Banking Services     56326
Name: product, dtype: int64

In [ ]:
import plotly.graph_objects as go

labels = df1['product'].value_counts().sort_index().index.to_list()
values = df1['product'].value_counts().sort_index().to_list()

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.38,title = 'Product distribution',sort = True, textinfo='percent')])
fig.show()

In [ ]:
#Display the 'complaint' of the first entry
df1.iloc[0]['complaint']

'transworld systems inc. \nis trying to collect a debt that is not mine, not owed and is inaccurate.'

In [ ]:
#Display the 'product' of the first entry
df1.iloc[0]['product']

'Debt Collection'

Create a function for formatting elapsed times as hh:mm:ss

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

Define the function **get_text_analysis** to analyze the <br><br>

1)  Words in the text<br>
2)  Number of words<br>
3)  Number of characters<br>
4)  Number of unique words<br>
5)  Number of alphabets<br>
6)  Number of numerical values<br>
7)  Number of white spaces<br>
8)  Number of other characters<br>
9)  Number of character 'X'<br>
10) Number of character 'x'<br>
11) Percentage of alphabets<br>
12) Percentage of numerical values<br>
13) Percentage of white spaces<br>
14) Percentage of other characters<br>
15) Percentage of character 'X'<br>
16) Percentage of character 'x'<br>


In [ ]:
import time 

pd.options.mode.chained_assignment = None
def get_text_analysis(df,column_name):
  '''
  df: the input dataframe containing column_name
  column_name : the column in df for which text analysis is to be done
  '''
  t0 = time.time()
  #Split column_name into substrings whenever whitespace occur
  df['split_words_whitespaces'] = df[column_name].map(lambda x: x.split())
  #Count the number of substrings in 'split_words_whitespaces'
  df['number_of_words'] = df['split_words_whitespaces'].map(lambda x: len(x))
  #Count the number of characters in column_name
  df['number_of_characters'] = df[column_name].map(lambda x: len(x))
  #Count the number of unique strings in 'split_words_whitespaces'
  df['number_of_unique_words'] = df['split_words_whitespaces'].map(lambda x : len(set(x)))
  #Count the number of alphabets in column_name
  df['number_of_alphabets'] = df[column_name].map(lambda x: len([c for c in x if c.isalpha()]))
  #Count the number of numerical values in column_name
  df['number_of_numerical_values'] = df[column_name].map(lambda x: len([c for c in x if c.isnumeric()]))
  #Count the number of white spaces in column_name
  df['number_of_white_spaces'] = df[column_name].map(lambda x: len([c for c in x if c.isspace()]) )
  #Count the number of other characters in column_name
  df['number_of_other_characters'] = df['number_of_characters'] - df['number_of_alphabets'] - df['number_of_numerical_values'] - df['number_of_white_spaces']
  #Count the number of character X in column_name
  df['number_of_character_X'] = df[column_name].map(lambda row: row.count('X'))
  #Count the number of character x in column_name
  df['number_of_character_x'] = df[column_name].map(lambda row: row.count('x'))
  #Calculate the percentage of alphabets in column_name
  df['percentage_of_alphabets'] = df['number_of_alphabets']/df['number_of_characters']
  #Calculate the percentage of numerical values in column_name
  df['percentage_of_numerical_values'] = df['number_of_numerical_values']/df['number_of_characters']
  #Calculate the percentage of white spaces in column_name
  df['percentage_of_white_spaces'] = df['number_of_white_spaces']/df['number_of_characters']
  #Calculate the percentage of other characters in column_name
  df['percentage_of_other_characters'] = df['number_of_other_characters']/df['number_of_characters']
  #Calculate the percentage of character X in column_name
  df['percentage_of_character_X'] = df['number_of_character_X']/df['number_of_characters']
  #Calculate the percentage of character x in column_name
  df['percentage_of_character_x'] = df['number_of_character_x']/df['number_of_characters']
  print("\nTime taken to get text metrics: {:} (h:mm:ss)\n\n".format(format_time(time.time() - t0)))
  df.info()


In [ ]:
#Define the function display_descriptive_statistics to display statistics like minimum, median, maximum and standard deviation
def display_descriptive_statistics(df):
  '''
  df: the dataframe for which the descriptive statistics like minimum, median, maximum and standard deviation is to be displayed
  '''
  return df.agg(['min','median','max','std']).T


In [ ]:
#Get the text analysis for the 'complaint' column in df1
get_text_analysis(df1,'complaint')


Time taken to get text metrics: 0:03:03 (h:mm:ss)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 652160 entries, 0 to 1922571
Data columns (total 19 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   product                         652160 non-null  object 
 1   complaint                       652160 non-null  string 
 2   original_product                652160 non-null  object 
 3   split_words_whitespaces         652160 non-null  object 
 4   number_of_words                 652160 non-null  int64  
 5   number_of_characters            652160 non-null  int64  
 6   number_of_unique_words          652160 non-null  int64  
 7   number_of_alphabets             652160 non-null  int64  
 8   number_of_numerical_values      652160 non-null  int64  
 9   number_of_white_spaces          652160 non-null  int64  
 10  number_of_other_characters      652160 non-null  int64  
 11  number_of_character_X   

In [ ]:
#Store the name of text metrics in a list 
text_metrics = [
                'number_of_words','number_of_characters', 'number_of_unique_words', 
                'number_of_alphabets', 'number_of_numerical_values',
                'number_of_white_spaces', 'number_of_other_characters',
                'number_of_character_X','number_of_character_x', 
                'percentage_of_alphabets', 'percentage_of_numerical_values',
                'percentage_of_white_spaces', 'percentage_of_other_characters',
                'percentage_of_character_X','percentage_of_character_x'
                ]

In [ ]:
#Display the descriptive statistics of text metrics in df1
#take snapshot
display_descriptive_statistics(df1[text_metrics])

,min,median,max,std
number_of_words,1.00000,128.000000,6314.000000,225.085042
number_of_characters,4.00000,705.000000,32317.000000,1265.504492
number_of_unique_words,1.00000,81.000000,1839.000000,81.663219
number_of_alphabets,4.00000,550.000000,25681.000000,990.486837
number_of_numerical_values,0.00000,4.000000,3259.000000,19.395898
number_of_white_spaces,0.00000,129.000000,7172.000000,230.217405
number_of_other_characters,0.00000,16.000000,5229.000000,42.432988
number_of_character_X,0.00000,21.000000,8136.000000,95.691499
number_of_character_x,0.00000,0.000000,150.000000,2.421816
percentage_of_alphabets,0.01922,0.786962,1.000000,0.030456


In [ ]:
# Retain certain records (with word count more than 7 and charachter count more than 14) only
df2 = df1.query('(number_of_characters > 14)&(number_of_words > 7)')
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 647953 entries, 0 to 1922571
Data columns (total 19 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   product                         647953 non-null  object 
 1   complaint                       647953 non-null  string 
 2   original_product                647953 non-null  object 
 3   split_words_whitespaces         647953 non-null  object 
 4   number_of_words                 647953 non-null  int64  
 5   number_of_characters            647953 non-null  int64  
 6   number_of_unique_words          647953 non-null  int64  
 7   number_of_alphabets             647953 non-null  int64  
 8   number_of_numerical_values      647953 non-null  int64  
 9   number_of_white_spaces          647953 non-null  int64  
 10  number_of_other_characters      647953 non-null  int64  
 11  number_of_character_X           647953 non-null  int64  
 12  number_of_chara

In [ ]:
# Display the descriptive statistics of text metrics in df2
display_descriptive_statistics(df2[text_metrics])

,min,median,max,std
number_of_words,8.000000,129.000000,6314.000000,225.304262
number_of_characters,30.000000,710.000000,32317.000000,1266.886754
number_of_unique_words,1.000000,82.000000,1839.000000,81.554683
number_of_alphabets,18.000000,554.000000,25681.000000,991.573878
number_of_numerical_values,0.000000,4.000000,3259.000000,19.442699
number_of_white_spaces,7.000000,130.000000,7172.000000,230.446830
number_of_other_characters,0.000000,17.000000,5229.000000,42.510997
number_of_character_X,0.000000,24.000000,8136.000000,95.926315
number_of_character_x,0.000000,0.000000,150.000000,2.427749
percentage_of_alphabets,0.019220,0.786828,0.973942,0.029788


In [ ]:
#Create a sample data sample_big which has 20000 entries of each of the product counts in df2
df2_group = df2.groupby('product', group_keys=False)
sample_big = df2_group.sample(n =20000, random_state = fix_seed, replace =False)
sample_big .info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120000 entries, 1597758 to 824502
Data columns (total 19 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   product                         120000 non-null  object 
 1   complaint                       120000 non-null  string 
 2   original_product                120000 non-null  object 
 3   split_words_whitespaces         120000 non-null  object 
 4   number_of_words                 120000 non-null  int64  
 5   number_of_characters            120000 non-null  int64  
 6   number_of_unique_words          120000 non-null  int64  
 7   number_of_alphabets             120000 non-null  int64  
 8   number_of_numerical_values      120000 non-null  int64  
 9   number_of_white_spaces          120000 non-null  int64  
 10  number_of_other_characters      120000 non-null  int64  
 11  number_of_character_X           120000 non-null  int64  
 12  number_of_

In [ ]:
# Display the descriptive statistics of text metrics in sample_big
display_descriptive_statistics(sample_big[text_metrics])

,min,median,max,std
number_of_words,8.000000,151.000000,5813.000000,239.951312
number_of_characters,30.000000,826.000000,31705.000000,1337.572385
number_of_unique_words,5.000000,94.000000,1642.000000,85.332625
number_of_alphabets,20.000000,642.000000,25580.000000,1044.595245
number_of_numerical_values,0.000000,5.000000,3259.000000,22.285141
number_of_white_spaces,7.000000,152.000000,5997.000000,244.889497
number_of_other_characters,0.000000,20.000000,5229.000000,46.625493
number_of_character_X,0.000000,24.000000,7792.000000,94.086056
number_of_character_x,0.000000,0.000000,110.000000,2.567328
percentage_of_alphabets,0.305976,0.784148,0.973942,0.028512


## Text Pre-processing using Gensim

In [ ]:
#Install the package folium version 0.2.1 and the package pattern (requirement for gensim library)
!pip install folium==0.2.1 pattern -q

     |████████████████████████████████| 69 kB 5.2 MB/s 
     |████████████████████████████████| 22.2 MB 1.4 MB/s 
     |████████████████████████████████| 87 kB 6.6 MB/s 
     |████████████████████████████████| 81 kB 9.1 MB/s 
     |████████████████████████████████| 5.6 MB 61.0 MB/s 
     |████████████████████████████████| 5.6 MB 66.6 MB/s 
     |████████████████████████████████| 419 kB 69.9 MB/s 
     |████████████████████████████████| 104 kB 74.4 MB/s 
     |████████████████████████████████| 3.6 MB 58.3 MB/s 


In [ ]:
# Import libraries for text processing
import string
import re
import pattern
from gensim.utils import tokenize

def text_pre_process(text):

  """
  text: the input text
  the function text_pre_process outputs preprocessed_text that is the pre-processed text
  """
  # Some basic helper functions to clean text.
  def remove_URL(text):
    '''
    text: input text for which the url has to be removed
    the function remove_URL removes the URL from the text
    '''
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

  def remove_emoji(text):
    '''
    text: input text for which the emoji has to be removed
    the function remove_emoji removes the emoji from the text
    '''
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
  
  def remove_html(text):
    '''
    text: input text for which the html has to be removed
    the function remove_html removes the html from the text
    '''
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)
   
  def remove_non_ascii(text):
    '''
    text: input text for which Non ASCII Characters has to be removed 
    the function remove_non_ascii removes the Non ASCII Characters from the text
    '''
    non_ascii_removed = text.encode('ascii', 'ignore').decode('ascii')
    return non_ascii_removed   

  def remove_character_X_and_x(text):
    '''
    text: input text for which more than one occurrence of character X or x has to be removed
    the function remove_character_X_and_x removes more than one occurrence of character X  or x from the text
    '''
    remove_X = re.sub(r'XX+','',text)
    remove_X_space = re.sub(r'X[/s]X','',remove_X )   
    remove_x = re.sub(r'xx+','',remove_X_space)
    remove_x_space = re.sub(r'x[/s]x','',remove_x)
    return remove_x_space

  def remove_punct(text):
    '''
    text: input text for which punctuations has to be removed 
    the function remove_punct removes the punctuations !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~  from the text
    '''
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)
  
  def remove_extra_white_spaces(text):
    '''
    text: input text for which the characters [\t\n\r\f\v] has to be replaced by a single space character
    the function remove_extra_white_spaces replaces the characters [\t\n\r\f\v] by a single space character
    '''
    #the characters [\t\n\r\f\v] are replaced by a single space character
    remove_extra_space_characters = re.sub('/s',' ',text)
    #replace more than one occurrence of a white space character by an occurrence
    remove_extra_space = re.sub(' +',' ',remove_extra_space_characters)
    return remove_extra_space
  
  def remove_consecutive_repeated_substrings(text):
    '''
    text: input text for which the consecutive repeated substring is to be removed
    the function remove_consecutive_repeated_substrings removes the consecutive repeated substring in text
    '''
    while re.search(r'\b(.+)(\s+\1\b)+', text):
      text = re.sub(r'\b(.+)(\s+\1\b)+', r'\1', text)
    return text
  
  # Applying helper functions

  #remove URL from the text
  text_clean = remove_URL(text)
  #remove emoji from text_clean
  text_clean = remove_emoji(text_clean)
  #remove html from text_clean
  text_clean =  remove_html(text_clean)
  #remove Non ASCII Characters from text_clean
  text_clean = remove_non_ascii(text_clean)  
  #remove more than one occurrence of character X and x from text_clean
  text_clean =  remove_character_X_and_x(text_clean)
  #remove punctuations from text_clean
  text_clean = remove_punct(text_clean)

  # Tokenizing 'text_clean' using tokenize function from gensim.utils 
  tokenized_data = list(tokenize(text_clean))

  # Lower casing 'tokenized_data'
  lower_text = str(' '.join([word.lower() for word in tokenized_data]))

  #Remove extra white spaces of lower_text
  extra_space_removed = remove_extra_white_spaces(lower_text)

  #Remove consecutive repeated substrings of extra_space_removed
  consecutive_repeated_substrings_removed = remove_consecutive_repeated_substrings(extra_space_removed)

  #Remove the quotes at the end of consecutive_repeated_substrings_removed
  preprocessed_text =  consecutive_repeated_substrings_removed.strip()
  return preprocessed_text

Example of text_pre_process function

In [ ]:
sample_big.loc[sample_big.index[0],'complaint']

"This is the third complaint I have filed against Wells Fargo. First they opened a credit card in my husband 's name and mine. We did not request this card. I also found out they attached the credit card to an account that was in my name since XXXX. This account was set up for my job. All direct deposits for pay went to this account. I also had set up this account for online bill pay. They changed the primary owner of this account to my husband so that the credit card that Wells Fargo opened ILLEGALLY, without our approval or knowledge and added it as overdraft for the the account. In XXXX XXXX, I made transfers for XXXX transactions from another checking account in my name to what I thought was my primary account to pay bills over a 35 day period. Not only did Wells Fargo transfer those monies back into the checking but they also made cash advances on the credit card and charged us for overdraft protection resulting in almost {$500.00} in cash advance fees, interest, and this overdraf

In [ ]:
text_pre_process(sample_big.loc[sample_big.index[0],'complaint'])

'this is the third complaint i have filed against wells fargo first they opened a credit card in my husband s name and mine we did not request this card i also found out they attached the credit card to an account that was in my name since this account was set up for my job all direct deposits for pay went to this account i also had set up this account for online bill pay they changed the primary owner of this account to my husband so that the credit card that wells fargo opened illegally without our approval or knowledge and added it as overdraft for the account in i made transfers for transactions from another checking account in my name to what i thought was my primary account to pay bills over a day period not only did wells fargo transfer those monies back into the checking but they also made cash advances on the credit card and charged us for overdraft protection resulting in almost in cash advance fees interest and this overdraft protection fee neither my husband or i knew that 

In [ ]:
# Obtain the 'preprocessed_text' for the 'complaint' data in sample_big
from tqdm import tqdm
tqdm.pandas()
t0 = time.time()
sample_big.loc[:,'preprocessed_text']= sample_big.loc[:,'complaint'].progress_map(lambda row: text_pre_process(row))
print("\nText Transformation Completed in {:} (h:mm:ss)\n\n".format(format_time(time.time() - t0)))

100%|██████████| 120000/120000 [1:42:57<00:00, 19.43it/s]


Text Transformation Completed in 1:42:57 (h:mm:ss)




In [ ]:
# Display the 'complaint' and 'preprocessed_text' for the first 3 rows
pd.set_option('display.max_colwidth', None)
#create a random sample check_preprocessing_sample of 50 entries from the dataframe sample_big 
check_preprocessing_sample = sample_big.sample(n = 50, random_state = fix_seed )
display(sample_big.loc[:,['complaint','preprocessed_text']][:3])

,complaint,preprocessed_text
1597758,"This is the third complaint I have filed against Wells Fargo. First they opened a credit card in my husband 's name and mine. We did not request this card. I also found out they attached the credit card to an account that was in my name since XXXX. This account was set up for my job. All direct deposits for pay went to this account. I also had set up this account for online bill pay. They changed the primary owner of this account to my husband so that the credit card that Wells Fargo opened ILLEGALLY, without our approval or knowledge and added it as overdraft for the the account. In XXXX XXXX, I made transfers for XXXX transactions from another checking account in my name to what I thought was my primary account to pay bills over a 35 day period. Not only did Wells Fargo transfer those monies back into the checking but they also made cash advances on the credit card and charged us for overdraft protection resulting in almost {$500.00} in cash advance fees, interest, and this overdraft protection fee. Neither my husband or I knew that my primary account had been manipulated and my identity and my husband 's were stolen by using our social security information. Now today XXXX/XXXX/XXXX, I have found my other credit card information has also been changed for the online bill pay lists placing all of my credit cards, different from my husbands and not in his name, changed to his name along with changing the account numbers for these cards. XXXX card number was not correct at all. I now have to check all of our credit card statements to make sure that the bills have been paid!!! My advice to any Wells Fargo account holder is to check everything in your accounts because they did a lot of damage not only in CA but across the US and in many different areas of your banking accounts!",this is the third complaint i have filed against wells fargo first they opened a credit card in my husband s name and mine we did not request this card i also found out they attached the credit card to an account that was in my name since this account was set up for my job all direct deposits for pay went to this account i also had set up this account for online bill pay they changed the primary owner of this account to my husband so that the credit card that wells fargo opened illegally without our approval or knowledge and added it as overdraft for the account in i made transfers for transactions from another checking account in my name to what i thought was my primary account to pay bills over a day period not only did wells fargo transfer those monies back into the checking but they also made cash advances on the credit card and charged us for overdraft protection resulting in almost in cash advance fees interest and this overdraft protection fee neither my husband or i knew that my primary account had been manipulated and my identity and my husband s were stolen by using our social security information now today i have found my other credit card information has also been changed for the online bill pay lists placing all of my credit cards different from my husbands and not in his name changed to his name along with changing the account numbers for these cards card number was not correct at all i now have to check all of our credit card statements to make sure that the bills have been paid my advice to any wells fargo account holder is to check everything in your accounts because they did a lot of damage not only in ca but across the us and in many different areas of your banking accounts
1290191,"This is a follow up of my original complaint case # XXXX XXXX . PNC Bak has failed to provide me with and explanation why {$360.00} was taken out of my personal checking account ending in XXXX on XXXX , XXXX XXXX . They have failed to provide me with the complete investigation file which PNC Bank claims I have mishandled my personal checking account. This request is specifically directed to a letter dated XXXX XXXX , XXXX

In [ ]:
# Set the display max_columns to 50
pd.pandas.set_option('display.max_columns', 50)
display(check_preprocessing_sample .loc[:,['complaint','preprocessed_text']])
# Set the display max_columns to 10
pd.pandas.set_option('display.max_columns', 10)

,complaint,preprocessed_text
799075,"In XXXX a valid debt was charged off from XXXX XXXX XXXX XXXX XXXX. I paid this in a settlement in XXXX over the phone to a collection agency. I contested this still being on my credit report in XXXX XXXX. I received a settlement letter on XXXX/XXXX/XXXX from XXXX XXXX XXXX on behalf of XXXX XXXX Bank. When I called XXXX on XXXX/XXXX/XXXX to say I had already settled this back in XXXX, but in the interest of settlement, I was willing to pay it again if I could be ensured it would be reflected properly on my credit report, they directed me to call Galaxy International Purchasing LLC ( XXXX ) to ensure it would reflect properly upon payment. After repeated calls, I finally got through to Galaxy on XXXX/XXXX/XXXX and explained the situation. They told me to call XXXX XXXX Bank ( XXXX ) as they have nothing to do with my credit report. I called XXXX XXXX Bank, after verifying my information and that it was in concern to a debt patched me through to some other company of which I did n't get their name due to a language barrier. The lady I spoke with told me to disregard the letter I received on XXXX from XXXX, despite XXXX telling me they were representing XXXX XXXX and that I had 60 days to accept their settlement offer. I was told XXXX XXXX has no record of them or Galaxy handling this account and that I should work through XXXX XXXX ( XXXX XXXX, a company I 've never received anything from. I have called XXXX XXXX and have n't been able to get through to them. When searching XXXX 's number it comes up as XXXX XXXX XXXX XXXX All of these companies, XXXX XXXX XXXX XXXX, Galaxy, they all have slews of scam warnings and complaints online for shady practices. This same thing happened in XXXX and my settlement payment was lost in the mix. This has already been paid off once and I fear that if I pay anyone in this matter it will only get lost again, especially since they all seem to think I can pay them and no one else. I want this account deleted from my credit report as it was paid XXXX years ago and I 'm being scammed in an endless loop of calling other scammy businesses.",in a valid debt was charged off from i paid this in a settlement in over the phone to a collection agency i contested this still being on my credit report in i received a settlement letter on from on behalf of bank when i called on to say i had already settled this back in but in the interest of settlement i was willing to pay it again if i could be ensured it would be reflected properly on my credit report they directed me to call galaxy international purchasing llc to ensure it would reflect properly upon payment after repeated calls i finally got through to galaxy on and explained the situation they told me to call bank as they have nothing to do with my credit report i called bank after verifying my information and that it was in concern to a debt patched me through to some other company of which i did nt get their name due to a language barrier the lady i spoke with told me to disregard the letter i received on from despite telling me they were representing and that i had days to accept their settlement offer i was told has no record of them or galaxy handling this account and that i should work through a company i ve never received anything from i have called and have nt been able to get through to them when searching s number it comes up as all of these companies galaxy they all have slews of scam warnings and complaints online for shady practices this same thing happened in and my settlement payment was lost in the mix this has already been paid off once and i fear that if i pay anyone in this matter it will only get lost again especially since they all seem to think i can pay them and no one else i want this account deleted from my credit report as it was paid years ago and i m being scammed in an endless loop of calling other scammy businesses
1645882,"When Collection agency first contacted me, I was only told that if

In [ ]:
# Get the text analysis for the 'preprocessed_text' column in check_preprocessing_sample
get_text_analysis(check_preprocessing_sample,'preprocessed_text')


Time taken to get text metrics: 0:00:00 (h:mm:ss)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 799075 to 527927
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   product                         50 non-null     object 
 1   complaint                       50 non-null     string 
 2   original_product                50 non-null     object 
 3   split_words_whitespaces         50 non-null     object 
 4   number_of_words                 50 non-null     int64  
 5   number_of_characters            50 non-null     int64  
 6   number_of_unique_words          50 non-null     int64  
 7   number_of_alphabets             50 non-null     int64  
 8   number_of_numerical_values      50 non-null     int64  
 9   number_of_white_spaces          50 non-null     int64  
 10  number_of_other_characters      50 non-null     int64  
 11  number_of_character_X           50 no

In [ ]:
# Display the descriptive statistics of text metrics in check_preprocessing_sample
display_descriptive_statistics(check_preprocessing_sample[text_metrics])

,min,median,max,std
number_of_words,8.000000,164.500000,1102.000000,170.231129
number_of_characters,69.000000,865.000000,6091.000000,919.902973
number_of_unique_words,5.000000,86.000000,312.000000,53.276713
number_of_alphabets,57.000000,700.000000,4990.000000,750.144649
number_of_numerical_values,0.000000,0.000000,0.000000,0.000000
number_of_white_spaces,7.000000,163.500000,1101.000000,170.231129
number_of_other_characters,0.000000,0.000000,0.000000,0.000000
number_of_character_X,0.000000,0.000000,0.000000,0.000000
number_of_character_x,0.000000,1.000000,7.000000,1.768315
percentage_of_alphabets,0.798194,0.816366,0.898551,0.016321


In [ ]:
# Display information of sample_big
sample_big.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120000 entries, 1597758 to 824502
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   product                         120000 non-null  object 
 1   complaint                       120000 non-null  string 
 2   original_product                120000 non-null  object 
 3   split_words_whitespaces         120000 non-null  object 
 4   number_of_words                 120000 non-null  int64  
 5   number_of_characters            120000 non-null  int64  
 6   number_of_unique_words          120000 non-null  int64  
 7   number_of_alphabets             120000 non-null  int64  
 8   number_of_numerical_values      120000 non-null  int64  
 9   number_of_white_spaces          120000 non-null  int64  
 10  number_of_other_characters      120000 non-null  int64  
 11  number_of_character_X           120000 non-null  int64  
 12  number_of_

In [ ]:
# Get the text analysis for the 'preprocessed_text' column in sample_big
get_text_analysis(sample_big,'preprocessed_text')


Time taken to get text metrics: 0:00:34 (h:mm:ss)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 120000 entries, 1597758 to 824502
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   product                         120000 non-null  object 
 1   complaint                       120000 non-null  string 
 2   original_product                120000 non-null  object 
 3   split_words_whitespaces         120000 non-null  object 
 4   number_of_words                 120000 non-null  int64  
 5   number_of_characters            120000 non-null  int64  
 6   number_of_unique_words          120000 non-null  int64  
 7   number_of_alphabets             120000 non-null  int64  
 8   number_of_numerical_values      120000 non-null  int64  
 9   number_of_white_spaces          120000 non-null  int64  
 10  number_of_other_characters      120000 non-null  int64  
 11  number_of_character

In [ ]:
# Display the descriptive statistics of text metrics in sample_big
display_descriptive_statistics(sample_big[text_metrics])

,min,median,max,std
number_of_words,1.000000,139.000000,5665.000000,217.995735
number_of_characters,7.000000,745.000000,30397.000000,1202.264097
number_of_unique_words,1.000000,82.000000,1171.000000,69.348835
number_of_alphabets,7.000000,607.000000,24793.000000,985.292748
number_of_numerical_values,0.000000,0.000000,0.000000,0.000000
number_of_white_spaces,0.000000,138.000000,5664.000000,217.995735
number_of_other_characters,0.000000,0.000000,0.000000,0.000000
number_of_character_X,0.000000,0.000000,0.000000,0.000000
number_of_character_x,0.000000,1.000000,193.000000,2.686710
percentage_of_alphabets,0.740741,0.815668,1.000000,0.013950


In [ ]:
# Retain certain records (with word count more than 7 and charachter count more than 14) only
sample_big_new = sample_big.query('(number_of_characters > 14)&(number_of_words > 7)')
sample_big_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119843 entries, 1597758 to 824502
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   product                         119843 non-null  object 
 1   complaint                       119843 non-null  string 
 2   original_product                119843 non-null  object 
 3   split_words_whitespaces         119843 non-null  object 
 4   number_of_words                 119843 non-null  int64  
 5   number_of_characters            119843 non-null  int64  
 6   number_of_unique_words          119843 non-null  int64  
 7   number_of_alphabets             119843 non-null  int64  
 8   number_of_numerical_values      119843 non-null  int64  
 9   number_of_white_spaces          119843 non-null  int64  
 10  number_of_other_characters      119843 non-null  int64  
 11  number_of_character_X           119843 non-null  int64  
 12  number_of_

In [ ]:
# Display the descriptive statistics of text metrics in sample_big_new
display_descriptive_statistics(sample_big_new[text_metrics])

,min,median,max,std
number_of_words,8.000000,139.000000,5665.000000,218.024963
number_of_characters,30.000000,746.000000,30397.000000,1202.452339
number_of_unique_words,4.000000,82.000000,1171.000000,69.313620
number_of_alphabets,23.000000,608.000000,24793.000000,985.452866
number_of_numerical_values,0.000000,0.000000,0.000000,0.000000
number_of_white_spaces,7.000000,138.000000,5664.000000,218.024963
number_of_other_characters,0.000000,0.000000,0.000000,0.000000
number_of_character_X,0.000000,0.000000,0.000000,0.000000
number_of_character_x,0.000000,1.000000,193.000000,2.688023
percentage_of_alphabets,0.740741,0.815657,0.995970,0.013822


In [ ]:
# Reset the index of sample_big_new and store the original indices in a new column and name than column as 'original_index'
sample_big_new= sample_big_new.reset_index()
sample_big_new.rename(columns={"index": "original_index"},inplace= True)
sample_big_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119843 entries, 0 to 119842
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   original_index                  119843 non-null  int64  
 1   product                         119843 non-null  object 
 2   complaint                       119843 non-null  string 
 3   original_product                119843 non-null  object 
 4   split_words_whitespaces         119843 non-null  object 
 5   number_of_words                 119843 non-null  int64  
 6   number_of_characters            119843 non-null  int64  
 7   number_of_unique_words          119843 non-null  int64  
 8   number_of_alphabets             119843 non-null  int64  
 9   number_of_numerical_values      119843 non-null  int64  
 10  number_of_white_spaces          119843 non-null  int64  
 11  number_of_other_characters      119843 non-null  int64  
 12  number_of_charac

In [ ]:
#Del Data to free memory
del raw_data ,df1,df2
import gc
gc.collect()

176

In [ ]:
# Group the dataframe sample_big_new by 'preprocessed_text' and store the entries as a list
sample_big_new_group_text = sample_big_new.groupby(sample_big_new['preprocessed_text']).agg(list).reset_index()
sample_big_new_group_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117131 entries, 0 to 117130
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   preprocessed_text               117131 non-null  object
 1   original_index                  117131 non-null  object
 2   product                         117131 non-null  object
 3   complaint                       117131 non-null  object
 4   original_product                117131 non-null  object
 5   split_words_whitespaces         117131 non-null  object
 6   number_of_words                 117131 non-null  object
 7   number_of_characters            117131 non-null  object
 8   number_of_unique_words          117131 non-null  object
 9   number_of_alphabets             117131 non-null  object
 10  number_of_numerical_values      117131 non-null  object
 11  number_of_white_spaces          117131 non-null  object
 12  number_of_other_characters    

In [ ]:
# Calculate the number of instances in the column 'product' in sample_big_new_group_text
sample_big_new_group_text['product_instance_length'] = sample_big_new_group_text['product'].map(len)
# Calculate the number of unique instance in the column 'product' in sample_big_new_group_text
sample_big_new_group_text['product_unique_instance_length'] = sample_big_new_group_text['product'].map(lambda row: len(set(row)))
sample_big_new_group_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117131 entries, 0 to 117130
Data columns (total 23 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   preprocessed_text               117131 non-null  object
 1   original_index                  117131 non-null  object
 2   product                         117131 non-null  object
 3   complaint                       117131 non-null  object
 4   original_product                117131 non-null  object
 5   split_words_whitespaces         117131 non-null  object
 6   number_of_words                 117131 non-null  object
 7   number_of_characters            117131 non-null  object
 8   number_of_unique_words          117131 non-null  object
 9   number_of_alphabets             117131 non-null  object
 10  number_of_numerical_values      117131 non-null  object
 11  number_of_white_spaces          117131 non-null  object
 12  number_of_other_characters    

In [ ]:
#Query the entries which has only one product instance
sample_big_new_unique_instance = sample_big_new_group_text.query('product_instance_length == 1')
sample_big_new_unique_instance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116138 entries, 0 to 117130
Data columns (total 23 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   preprocessed_text               116138 non-null  object
 1   original_index                  116138 non-null  object
 2   product                         116138 non-null  object
 3   complaint                       116138 non-null  object
 4   original_product                116138 non-null  object
 5   split_words_whitespaces         116138 non-null  object
 6   number_of_words                 116138 non-null  object
 7   number_of_characters            116138 non-null  object
 8   number_of_unique_words          116138 non-null  object
 9   number_of_alphabets             116138 non-null  object
 10  number_of_numerical_values      116138 non-null  object
 11  number_of_white_spaces          116138 non-null  object
 12  number_of_other_characters    

In [ ]:
#Unlist the entries in sample_big_new_unique_instance
sample_big_new_instance_unlist = sample_big_new_unique_instance.applymap(lambda row : row[0] if isinstance(row, list) else row)
sample_big_new_instance_unlist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116138 entries, 0 to 117130
Data columns (total 23 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   preprocessed_text               116138 non-null  object 
 1   original_index                  116138 non-null  int64  
 2   product                         116138 non-null  object 
 3   complaint                       116138 non-null  object 
 4   original_product                116138 non-null  object 
 5   split_words_whitespaces         116138 non-null  object 
 6   number_of_words                 116138 non-null  int64  
 7   number_of_characters            116138 non-null  int64  
 8   number_of_unique_words          116138 non-null  int64  
 9   number_of_alphabets             116138 non-null  int64  
 10  number_of_numerical_values      116138 non-null  int64  
 11  number_of_white_spaces          116138 non-null  int64  
 12  number_of_other_

In [ ]:
#Calculate the product count in sample_big_new_instance_unlist
sample_big_new_instance_unlist['product'].value_counts()

Mortgage            19960
Banking Services    19922
Loans               19825
Card Services       19583
Debt Collection     19333
Credit Reporting    17515
Name: product, dtype: int64

In [ ]:
#Query the entries which has more than one product instance and only one unique product instance
sample_big_new_unique_product_not_instance = sample_big_new_group_text.query('(product_unique_instance_length == 1) &(product_instance_length > 1)')
sample_big_new_unique_product_not_instance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 912 entries, 36 to 117084
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   preprocessed_text               912 non-null    object
 1   original_index                  912 non-null    object
 2   product                         912 non-null    object
 3   complaint                       912 non-null    object
 4   original_product                912 non-null    object
 5   split_words_whitespaces         912 non-null    object
 6   number_of_words                 912 non-null    object
 7   number_of_characters            912 non-null    object
 8   number_of_unique_words          912 non-null    object
 9   number_of_alphabets             912 non-null    object
 10  number_of_numerical_values      912 non-null    object
 11  number_of_white_spaces          912 non-null    object
 12  number_of_other_characters      912 non-null  

In [ ]:
#Unlist the entries in sample_big_new_product_not_instance_unlist and choose the first instance
sample_big_new_product_not_instance_unlist = sample_big_new_unique_product_not_instance.applymap(lambda row : row[0] if isinstance(row, list) else row)
sample_big_new_product_not_instance_unlist .info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 912 entries, 36 to 117084
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   preprocessed_text               912 non-null    object 
 1   original_index                  912 non-null    int64  
 2   product                         912 non-null    object 
 3   complaint                       912 non-null    object 
 4   original_product                912 non-null    object 
 5   split_words_whitespaces         912 non-null    object 
 6   number_of_words                 912 non-null    int64  
 7   number_of_characters            912 non-null    int64  
 8   number_of_unique_words          912 non-null    int64  
 9   number_of_alphabets             912 non-null    int64  
 10  number_of_numerical_values      912 non-null    int64  
 11  number_of_white_spaces          912 non-null    int64  
 12  number_of_other_characters      

In [ ]:
#Concatenate the dataframes sample_big_new_instance_unlist and sample_big_new_product_not_instance_unlist
sample_big_new1 = pd.concat([sample_big_new_instance_unlist, sample_big_new_product_not_instance_unlist], ignore_index=True)
sample_big_new1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117050 entries, 0 to 117049
Data columns (total 23 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   preprocessed_text               117050 non-null  object 
 1   original_index                  117050 non-null  int64  
 2   product                         117050 non-null  object 
 3   complaint                       117050 non-null  object 
 4   original_product                117050 non-null  object 
 5   split_words_whitespaces         117050 non-null  object 
 6   number_of_words                 117050 non-null  int64  
 7   number_of_characters            117050 non-null  int64  
 8   number_of_unique_words          117050 non-null  int64  
 9   number_of_alphabets             117050 non-null  int64  
 10  number_of_numerical_values      117050 non-null  int64  
 11  number_of_white_spaces          117050 non-null  int64  
 12  number_of_other_

In [ ]:
#Display the descriptive statistics of text metrics in sample_big_new1
display_descriptive_statistics(sample_big_new1[text_metrics])

,min,median,max,std
number_of_words,8.000000,142.000000,5665.000000,217.960319
number_of_characters,30.000000,759.000000,30397.000000,1199.121585
number_of_unique_words,4.000000,83.000000,1171.000000,69.183741
number_of_alphabets,23.000000,618.000000,24793.000000,982.154171
number_of_numerical_values,0.000000,0.000000,0.000000,0.000000
number_of_white_spaces,7.000000,141.000000,5664.000000,217.960319
number_of_other_characters,0.000000,0.000000,0.000000,0.000000
number_of_character_X,0.000000,0.000000,0.000000,0.000000
number_of_character_x,0.000000,1.000000,193.000000,2.710422
percentage_of_alphabets,0.740741,0.815498,0.995970,0.013585


In [ ]:
#Calculate the product count in sample_big_new1
sample_big_new1['product'].value_counts()

Mortgage            19966
Banking Services    19942
Loans               19848
Card Services       19639
Debt Collection     19479
Credit Reporting    18176
Name: product, dtype: int64

In [ ]:
#Create a sample data sample_df which has 10000 entries of each of the product counts in sample_big_new1
gensim_data_df_group = sample_big_new1.groupby('product', group_keys=False)
sample_df = gensim_data_df_group.sample(n =10000, random_state = fix_seed, replace =False)
# Reset the index of sample_df
sample_df = sample_df.reset_index(drop=True)
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   preprocessed_text               60000 non-null  object 
 1   original_index                  60000 non-null  int64  
 2   product                         60000 non-null  object 
 3   complaint                       60000 non-null  object 
 4   original_product                60000 non-null  object 
 5   split_words_whitespaces         60000 non-null  object 
 6   number_of_words                 60000 non-null  int64  
 7   number_of_characters            60000 non-null  int64  
 8   number_of_unique_words          60000 non-null  int64  
 9   number_of_alphabets             60000 non-null  int64  
 10  number_of_numerical_values      60000 non-null  int64  
 11  number_of_white_spaces          60000 non-null  int64  
 12  number_of_other_characters      

In [ ]:
#Calculate the product count in sample_df
sample_df['product'].value_counts()

Banking Services    10000
Card Services       10000
Credit Reporting    10000
Debt Collection     10000
Loans               10000
Mortgage            10000
Name: product, dtype: int64

In [ ]:
#Display the descriptive statistics of text metrics in sample_df
display_descriptive_statistics(sample_df[text_metrics])

,min,median,max,std
number_of_words,8.000000,141.000000,5383.000000,219.892735
number_of_characters,30.000000,757.000000,30148.000000,1212.793377
number_of_unique_words,5.000000,83.000000,1171.000000,69.511275
number_of_alphabets,23.000000,617.000000,24766.000000,993.897519
number_of_numerical_values,0.000000,0.000000,0.000000,0.000000
number_of_white_spaces,7.000000,140.000000,5382.000000,219.892735
number_of_other_characters,0.000000,0.000000,0.000000,0.000000
number_of_character_X,0.000000,0.000000,0.000000,0.000000
number_of_character_x,0.000000,1.000000,193.000000,2.707624
percentage_of_alphabets,0.740741,0.815534,0.995970,0.013624


In [ ]:
#Display the first five rows in sample_df
pd.reset_option('^display.', silent=True)
sample_df.head()

,preprocessed_text,original_index,product,complaint,original_product,split_words_whitespaces,number_of_words,number_of_characters,number_of_unique_words,number_of_alphabets,...,number_of_character_X,number_of_character_x,percentage_of_alphabets,percentage_of_numerical_values,percentage_of_white_spaces,percentage_of_other_characters,percentage_of_character_X,percentage_of_character_x,product_instance_length,product_unique_instance_length
0,on i notified usbank fraud dept line of an una...,755334,Banking Services,On XX/XX/XXXX I notified USbank fraud dept lin...,Checking or savings account,"[on, i, notified, usbank, fraud, dept, line, o...",644,3777,300,3134,...,0,0,0.829759,0.0,0.170241,0.0,0.0,0.0,1,1
1,previously i had repeatedly received spam mail...,693962,Banking Services,"Previously, I had repeatedly received spam mai...","Money transfer, virtual currency, or money ser...","[previously, i, had, repeatedly, received, spa...",62,340,44,279,...,0,0,0.820588,0.0,0.179412,0.0,0.0,0.0,1,1
2,on i received a notice from that my car paymen...,248080,Banking Services,On XX/XX/19 I received a notice from XXXX XXXX...,Checking or savings account,"[on, i, received, a, notice, from, that, my, c...",420,2242,187,1823,...,0,0,0.813113,0.0,0.186887,0.0,0.0,0.0,1,1
3,on i initiated a transfer of from my bank acco...,1562368,Banking Services,"On XXXX XXXX, 2016 I initiated a transfer of {...",Money transfers,"[on, i, initiated, a, transfer, of, from, my, ...",201,1100,84,900,...,0,0,0.818182,0.0,0.181818,0.0,0.0,0.0,1,1
4,hi i used paypal to purchase eyelashes online ...,1637582,Banking Services,Hi I used Paypal to purchase eyelashes online ...,Money transfers,"[hi, i, used, paypal, to, purchase, eyelashes,...",101,544,67,444,...,0,0,0.816176,0.0,0.183824,0.0,0.0,0.0,1,1


## Download data

In [ ]:
sample_df.to_csv(saved_path +'data_sample_balanced.csv',index=False)
print("\nDownload Complete : data_sample_balanced.csv")


Download Complete : data_sample_balanced.csv


In [ ]:
sample_big_new1.to_csv(saved_path + 'sample_big_new1.csv',index=False)
print("\nDownload Complete : sample_big_new1.csv")


Download Complete : sample_big_new1.csv
